In [ ]:
import tensorflow as tf
import numpy as np
from osgeo import gdal
import os
import random
import pickle
import subprocess
import glob

In [ ]:
import sys
sys.path.insert(0, '/exch/scripts/sampler')
from Sampler import batchload

In [ ]:
WORKDIR = '/exch/gdrive/oilp_rubber'
SESSIONDIR = '/exch/scripts/tf_sess'
boarddir_train = '/exch/scripts/tb_logs/train/310'
boarddir_test = '/exch/scripts/tb_logs/test/310'
os.chdir(WORKDIR)

In [ ]:
#subprocess.Popen(['tensorboard', '--logdir=/exch/tb_logs', '--port=6006'])
learning_rate = 0.0001
num_steps = 500
batch_size = 8
#display_step = 10
num_epochs = 10000

# Network Parameters
edgelength = 32
#num_pixels = edgelength * edgelength
num_chans = 6
#num_input = num_pixels * num_chans # MNIST data input (img shape: 28*28)
num_classes = 5 # MNIST total classes (0-9 digits)
dropout = .85 # Dropout, probability to keep units
fac=2

# tf Graph input
with tf.name_scope('Input'):
    X = tf.placeholder(tf.float32, [None, edgelength, edgelength, num_chans])
    tf.summary.image('input_images', X[:, :, :, 0:3][:, :, :, ::-1], max_outputs=4)
    Y = tf.placeholder(tf.int32, [None, num_classes])#num_classes])
    #Y_oh = tf.one_hot(Y, 8)
    keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)
    phase_t = tf.placeholder(tf.bool)

In [ ]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1, phase=0):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    x = tf.layers.batch_normalization(x, training=phase)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout, phase_t):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    # x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], phase=phase_t)
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], phase=phase_t)
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], phase=phase_t)
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    
    # Convolution Layer
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'], phase=phase_t)
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    
    # Convolution Layer
    #conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    #conv5 = maxpool2d(conv5, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv4, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, num_chans, int(32*fac)])),
    'wc2': tf.Variable(tf.random_normal([3, 3, int(32*fac), int(64*fac)])),
    'wc3': tf.Variable(tf.random_normal([3, 3, int(64*fac), int(128*fac)])),
    'wc4': tf.Variable(tf.random_normal([3, 3, int(128*fac), int(256*fac)])),
    #'wc5': tf.Variable(tf.random_normal([3, 3, 256, 512])),
    
    'wd1': tf.Variable(tf.random_normal([2*2*int(256*fac), int(512*fac)])),
    'out': tf.Variable(tf.random_normal([int(512*fac), num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([int(32*fac)])),
    'bc2': tf.Variable(tf.random_normal([int(64*fac)])),
    'bc3': tf.Variable(tf.random_normal([int(128*fac)])),
    'bc4': tf.Variable(tf.random_normal([int(256*fac)])),
    #'bc5': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([int(512*fac)])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob, phase_t)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer

#Notice - we introduce our L2 here
vars_to_regul   = tf.trainable_variables() 
lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in vars_to_regul
                    if 'bias' not in v.name ]) * 0.001

#  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    out_layer, tf_train_labels) +
#    beta*tf.nn.l2_loss(hidden_weights) +
#    beta*tf.nn.l2_loss(hidden_biases) +
#    beta*tf.nn.l2_loss(out_weights) +
#    beta*tf.nn.l2_loss(out_biases)))

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y)) + lossL2


optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    #train_op = optimizer.minimize(loss)
    train_op = optimizer.minimize(loss_op)



######train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss_op)
merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(boarddir_train)
test_writer = tf.summary.FileWriter(boarddir_test)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
X_batches, y_batches, xte, yte, MNs, SDs = batchload(WORKDIR, 0.8, num_classes, batch_size=8, edgelength=32, centr=True)

In [ ]:
xte = xte[:32]
#yte = np.expand_dims(yte[:8],1)
a = np.array(yte[:32]-1)
b = np.zeros((32, num_classes))
b[np.arange(32), a] = 1
yte = b

In [ ]:
# Start training
# step = 0
display_step = 5
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    
    for e in range(1, num_epochs + 1):
        for i, xb in enumerate(X_batches):
            yb = y_batches[i]
            #yb = np.expand_dims(yb,1)            
            a = np.array(yb-1)
            b = np.zeros((8, num_classes))
            b[np.arange(8), a] = 1
            yb = b
            
            sess.run(train_op, feed_dict={X: xb, Y: yb, keep_prob: dropout, phase_t: 1}) 
            #print(i)
            
        if e % display_step == 0 or e == 1:
            summary, loss, acc = sess.run([merged, loss_op, accuracy], feed_dict={X: xb,
                                                                 Y: yb,
                                                                 keep_prob: 1.0,
                                                                 phase_t: 0})
            train_writer.add_summary(summary, e)

            print("Step " + str(e) + ", Minibatch Loss= " + \
                    "{:.4f}".format(loss) + ", Training Accuracy= " + \
                    "{:.3f}".format(acc))

            # Calculate accuracy for 256 MNIST test images
            print("Testing Accuracy:")
            summary, acc = sess.run([merged, accuracy], feed_dict={X: xte,
                                      Y: yte,
                                      keep_prob: 1.0,
                                      phase_t: 0})
            test_writer.add_summary(summary, e)
            print(acc)
            
            save_path = saver.save(sess, SESSIONDIR+"/model.ckpt")
            print("Model saved in file: %s" % save_path)

In [ ]:
S = gdal.Open('/exch/gdrive/newhm_again.tif')
Xr = S.ReadAsArray()
where_are_NaNs = np.isnan(Xr)
Xr[where_are_NaNs] = -40.

Z = [(Xr[i,:,:] - MNs[i])/SDs[i] for i in range(len(MNs))]
Z = np.array(Z)
Z = np.transpose(Z)
#M = np.zeros(Xr[:,:,0].shape)

M = np.zeros(Z[:,:,0].shape)

In [ ]:
saver = tf.train.Saver()

yb = y_batches[2]-1       
a = np.array(yb)
b = np.zeros((8, num_classes))
b[np.arange(8), a] = 1
yb = b

with tf.Session() as sess:
    saver.restore(sess, SESSIONDIR+"/model.ckpt")
    
    for i in range(18,Z.shape[0]-18):
        for j in range(18,Z.shape[1]-18):
            #print(y)
            classify_this = Z[i-int(edgelength/2):i+int(edgelength/2),j-int(edgelength/2):j+int(edgelength/2),:]
            ct = np.expand_dims(classify_this, axis=0)           
            y_hat = sess.run(prediction, feed_dict={X: ct, Y: yb, keep_prob:1.0, phase_t: 0})#np.expand_dims(yb[0], axis=0), keep_prob:1.0})
            CLASS = np.argmax(y_hat)
            M[i,j] = CLASS
            #print(CLASS)
        print(i)

In [ ]:
# Write output
#S2 = gdal.Open('/data/Hdd1/CNNTEST.tif')
S2 = S
Xr2 = S2.ReadAsArray()
driver = gdal.GetDriverByName('Gtiff')
dstfile = WORKDIR+'/oil_rubber_map_3.tif'
dataset2 = driver.Create(dstfile, S2.RasterXSize, S2.RasterYSize, 1, gdal.GDT_Float32)
dataset2.SetGeoTransform(S2.GetGeoTransform())
dataset2.SetProjection(S2.GetProjection())
Mt = M.transpose()
dataset2.GetRasterBand(1).WriteArray(Mt)
dataset2.FlushCache()
dataset2 = None